- docs: https://face-recognition.readthedocs.io/en/latest/face_recognition.html
- face recognition used: https://github.com/ageitgey/face_recognition

## For testing only

In [8]:
import cv2
import face_recognition
""" print(cv2.__version__) """
# yall gottta add ur own images and change the path to the image if u wanna test too.
""" img = cv2.imread('faces/ryan_reynolds.jpg')
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_encodings = face_recognition.face_encodings(rgb_img)[0]

img2 = cv2.imread('faces/ryan_reynolds1.jpg')
rgb_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img_encodings2 = face_recognition.face_encodings(rgb_img2)[0]

img3 = cv2.imread('faces/ryan_gosling.webp')
rgb_img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
img_encodings3 = face_recognition.face_encodings(rgb_img3)[0]

img4 = cv2.imread('faces/ryan_gosling1.webp')
rgb_img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)
img_encodings4 = face_recognition.face_encodings(rgb_img4)[0] """

img = cv2.imread('faces/me.JPG')
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_encodings = face_recognition.face_encodings(rgb_img)[0]
cv2.imshow('Me', img)
cv2.waitKey(0)

""" cv2.imshow('Ryan Reynolds', img)
cv2.imshow('Ryan Reynolds 1', img2)
cv2.imshow('Ryan Gosling', img3)
cv2.waitKey(0) """

print(img_encodings)

RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

In [3]:
result = face_recognition.compare_faces([img_encodings], img_encodings2) # Should return True
print(result)

[np.True_]


In [4]:
result = face_recognition.compare_faces([img_encodings3], img_encodings4) # Should return True
print(result)

[np.True_]


In [5]:
result = face_recognition.compare_faces([img_encodings], img_encodings3) # Should return False
print(result)

[np.False_]


In [6]:
result = face_recognition.compare_faces([img_encodings2], img_encodings4) # Should return False
print(result)

[np.False_]


## Using real time video feed using webcam
- create a folder called faces
- add your image to the faces folder and pictures of other people to test

In [2]:
import cv2
import face_recognition
import numpy as np
import math
import os, sys

# Define face_confidence function
def face_confidence(face_distance, face_match_threshold=0.6):
    if face_distance > face_match_threshold:
        range = (1.0 - face_match_threshold)
        linear_val = (1.0 - face_distance) / (range * 2.0)
        return round(linear_val,2) #rounding to 2 decimal places
    else:
        range = face_match_threshold
        linear_val = 1.0 - (face_distance / (range * 2.0))
        return round(linear_val,2)
    
class FaceRecognition:
    face_locations = []
    face_encodings = []
    face_names = []
    known_face_encodings = []
    known_face_names = []
    process_current_frame = True #Reccommended from docs saves processing power not making it detect faces for every frame

    def __init__(self):
        self.encode_faces()

    def encode_faces(self):
        for image in os.listdir('faces'):
            face_image = face_recognition.load_image_file(f'faces/{image}')
            face_enc = face_recognition.face_encodings(face_image)[0]

            self.known_face_encodings.append(face_enc)
            self.known_face_names.append(image)
        print(self.known_face_names)

    def video_recognition(self):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Error: Could not open video.')
            sys.exit()

        while True:
            ret, frame = cap.read()

            if self.process_current_frame:
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

                # Find face in frame
                self.face_locations = face_recognition.face_locations(rgb_small_frame)
                self.face_encodings = face_recognition.face_encodings(rgb_small_frame, self.face_locations)

                #recognition part
                self.face_names = []
                for face_encoding in self.face_encodings:
                    """ match = face_recognition.compare_faces(self.known_face_encodings, face_encoding, tolerance=0.5) """ ## tolerance is the threshold for face matching. This is just for testing
                    match = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
                    # default name and confidence
                    name = '???'
                    confidence = '???'

                    face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)

                    if match[best_match_index]:
                        name = self.known_face_names[best_match_index]
                        confidence = face_confidence(face_distances[best_match_index])
                    
                    self.face_names.append(f'{name} ({confidence})')

            self.process_current_frame = not self.process_current_frame #only process every other frame

            # Drawing the binding box
            for(top, right, bottom, left), name in zip(self.face_locations, self.face_names):
                top*=4
                right*=4
                bottom*=4
                left*=4

                cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
                """ cv2.rectangle(frame, (left, bottom-35), (right, bottom), (255, 0, 0), -1) # """
                cv2.putText(frame, name, (left+6, bottom-6), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
            
            cv2.imshow('Face Recognition', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()



if __name__ == '__main__':
    fr = FaceRecognition()
    fr.video_recognition()

['me.JPG', 'ryan_gosling.webp', 'ryan_gosling1.webp', 'ryan_reynolds.jpg', 'ryan_reynolds1.jpg']


## DLIB
- For wheel issues go here to get the wheel https://github.com/z-mahmud22/Dlib_Windows_Python3.x
- After getting the wheel pip install it into ur env
- pip install --upgrade dlib afterwards. Somehow only 19.24.6 works for me

In [1]:
import dlib
print(dlib.__version__) # 19.22.0

19.24.6


## Functions to generating, saving and loading embedded face data for model

In [1]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
import pickle
import time


# Load pre-trained models
detector = dlib.get_frontal_face_detector() # This is a model to detect faces. Does not do the recognising part. Only detects the location of faces in the frame
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") # This model finds landmarks in the face like eyes, nose, lips. Used for embedding
face_rec_model = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat") # This model is used to compute the face embeddings to compare faces

# Function to compute the 128D face embedding
def get_face_embedding(face_image, shape):
    face_embedding = face_rec_model.compute_face_descriptor(face_image, shape)
    return np.array(face_embedding)

# Function to load an image, detect a face, and compute the embedding
def get_embedding_from_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Could not load image from {image_path}")
        return None, None

    print(f"Image dtype: {image.dtype}, shape: {image.shape}")

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    print(f"Gray image dtype: {gray.dtype}, shape: {gray.shape}")

    # Detect faces in the grayscale image
    faces = detector(gray)
    print(f"Number of faces detected: {len(faces)}")

    if len(faces) == 0:
        print("No face found in the image.")
        return None, None

    # Get the shape of the first detected face
    face = faces[0]
    shape = predictor(gray, face)

    # Get the embedding for the face
    embedding = get_face_embedding(image, shape)

    return embedding, image

# Function to compare two face embeddings using Euclidean distance
def compare_faces(known_embedding, new_embedding, threshold=0.6):
    distance = np.linalg.norm(known_embedding - new_embedding)
    return distance < threshold  # Return True if the distance is below the threshold

# List to store known face embeddings and their corresponding names
known_face_embeddings = []
known_face_names = []

# Function to add a new person by uploading an image and getting their embedding
def add_new_face(image_path, name):
    embedding, image = get_embedding_from_image(image_path)
    if embedding is not None:
        known_face_embeddings.append(embedding)
        known_face_names.append(name)
        print(f"Added {name} to the known faces database.")
        save_embeddings_to_pickle("face_embeddings.pkl")
        

# Function to load embeddings and names from a pickle file
def load_embeddings_from_pickle(pickle_file):
    global known_face_embeddings, known_face_names
    try:
        with open(pickle_file, 'rb') as f:
            known_face_embeddings, known_face_names = pickle.load(f)
        print(f"Loaded embeddings and names from {pickle_file}")
    except FileNotFoundError:
        print(f"Pickle file {pickle_file} not found.")
    except Exception as e:
        print(f"Error loading pickle file: {e}")

# Function to save embeddings and names to a pickle file
def save_embeddings_to_pickle(pickle_file):
    with open(pickle_file, 'wb') as f:
        pickle.dump((known_face_embeddings, known_face_names), f)
    print(f"Saved embeddings and names to {pickle_file}")

## UNCOMMENT THIS PART TO ADD NEW FACE
# I commented this part out to test the loading of embeddings from a pickle file
image_path = "faces/me.JPG"  
name = "Me"  
add_new_face(image_path, name)

# Load embeddings from a previously saved pickle file (if available)
pickle_file = "face_embeddings.pkl"
load_embeddings_from_pickle(pickle_file)



Image dtype: uint8, shape: (514, 400, 3)
Gray image dtype: uint8, shape: (514, 400)
Number of faces detected: 1
Added Me to the known faces database.
Saved embeddings and names to face_embeddings.pkl
Loaded embeddings and names from face_embeddings.pkl


## Function for running real time face detection

In [4]:
# Function to run face recognition on a live webcam feed
def run_face_recognition():
    min_face_size = 300 # Minimum bounding box size to detect a face. Used to filter out people who are far away
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    while True:
        # Capture frame-by-frame from the webcam
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = detector(gray, 1)
        for face in faces:
            x, y, w, h = face.left(), face.top(), face.right(), face.bottom()
            print(f"Face found at X: {x}, Y: {y}, W: {w}, H: {h}")
            if w >= min_face_size and h >= min_face_size:
                start_time = time.time()
                shape = predictor(gray, face)# Get the landmarks/parts for the face
                new_face_embedding = get_face_embedding(frame, shape)# Compute the face embedding for the detected face
            
                cv2.rectangle(frame, (x, y), (w, h), (0, 255, 0), 2)
                # Compare the new face embedding with known faces
                recognized_name = "Unknown"
                for i, known_embedding in enumerate(known_face_embeddings):
                    if compare_faces(known_embedding, new_face_embedding):
                        recognized_name = known_face_names[i]
                        break
                print(f"Recognised {recognized_name} in {time.time() - start_time:.8f} seconds")

                # Bounding box
                cv2.putText(frame, recognized_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

        # Display the resulting frame with face detections and names
        cv2.imshow('Video', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture when done
    cap.release()
    cv2.destroyAllWindows()

In [3]:
run_face_recognition()

Face found at X: 210, Y: 139, W: 339, H: 268
Face found at X: 253, Y: 133, W: 408, H: 288
Face found at X: 236, Y: 133, W: 390, H: 288
Face found at X: 201, Y: 116, W: 356, H: 270
Face found at X: 184, Y: 116, W: 339, H: 270
Face found at X: 184, Y: 133, W: 339, H: 288
Face found at X: 201, Y: 133, W: 356, H: 288
Face found at X: 201, Y: 133, W: 356, H: 288
Face found at X: 225, Y: 139, W: 354, H: 268
Face found at X: 218, Y: 133, W: 373, H: 288
Face found at X: 225, Y: 153, W: 354, H: 282
Face found at X: 201, Y: 150, W: 356, H: 305
Face found at X: 201, Y: 150, W: 356, H: 305
Face found at X: 225, Y: 153, W: 354, H: 282
Face found at X: 225, Y: 153, W: 354, H: 282
Face found at X: 201, Y: 150, W: 356, H: 305
Face found at X: 201, Y: 150, W: 356, H: 305
Face found at X: 201, Y: 133, W: 356, H: 288
Face found at X: 201, Y: 133, W: 356, H: 288
Face found at X: 201, Y: 133, W: 356, H: 288
Face found at X: 201, Y: 150, W: 356, H: 305
Face found at X: 201, Y: 150, W: 356, H: 305
Face found

## Testing for saving the face embeddings for future uses in database

In [6]:
import pickle

with open('face_embeddings.pkl', 'wb') as f:
    pickle.dump((known_face_embeddings, known_face_names), f)

print("Face embeddings and names saved successfully.")



Face embeddings and names saved successfully.


In [7]:
import pickle

# Load the embeddings and names from the .pkl file
with open('face_embeddings.pkl', 'rb') as f:
    known_face_embeddings, known_face_names = pickle.load(f)

print("Face embeddings and names loaded successfully.")
print(known_face_names) #for checking


Face embeddings and names loaded successfully.
['Me']
